# Problem
**Mục tiêu: Sử dụng các yếu tố sức khỏe để dự đoán một người có mắc bệnh tim hay không**

**Metric: AUC, Precision với Recall = 90 %**

Biến mục tiêu:

Có hai hướng đi:
1. Tình trạng bệnh tim với `0` là không mắc bệnh, `1` là mắc bệnh
2. Khả năng một người mắc bệnh tim (số thực mang giá trị `[0;1]`)
- **Important**: dataset của chúng ta không có sẵn biến mục tiêu dạng này, biến gần nhất là tình trạng bệnh tim nhận giá trị `0` hoặc `1`
- Đầu ra của mô hình phải phù hợp với các tính toán được y học chấp nhận

Thông qua phân tích sơ bộ, tỷ lệ mắc bệnh chung vào khoảng 9%, để đầu ra thực sự có ích, thì với recall 90% tôi mong muốn có thể có Precision tối thiểu 18%, và với recall 95%, precision 14%.

Các biến dự đoán: vì đây là một vấn đề cần phân tích kỹ hơn, nên tôi sẽ thực hiện file `preprocessing.ipynb` trước khi đưa ra những nhận định về các biến dự đoán.

Tuy chúng ta không thể tự mình tìm kiếm thêm các thông tin về yếu tố sức khỏe, nhưng việc xem xét thêm các yếu tố nằm ngoài bộ dataset được chứng minh bằng các nghiên cứu y học có thể giúp ích cho việc cải tiến mô hình sau này. Vì thế, hãy xem qua một số nguồn:
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7210118/
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7871691/

